### IS Proof of concept - DRMABS

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import itertools
import math
import string
import re

In [2]:
import time
from datetime import datetime
today = datetime.now()

In [3]:
import nltk
from nltk import bigrams
import collections

In [4]:
#pip install pythainlp
import pythainlp
from pythainlp import Tokenizer
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp import sent_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.util import dict_trie

In [5]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [6]:
# pip install selenium
from selenium import webdriver

In [7]:
from IPython.display import clear_output

In [75]:
# Files definition
root_path = 'Datasource'
urls_past_file,comment_past_file = [],[]
urls_past_file.append(root_path+"/urls_meiji.csv")
urls_past_file.append(root_path+"/urls_dutchmilk.csv")
urls_past_file.append(root_path+"/urls_dairyhome.csv")
urls_past_file.append(root_path+"/urls_chokchai.csv")
urls_past_file.append(root_path+"/urls_foremost.csv")

comment_past_file.append(root_path+"/comment_DDCF.csv")

# จำนวนกระทู้ที่ต้องการให้ดึงใน 1 นาที
postNum = 7
postNumSleep = math.ceil(60/postNum)

### กรณียังไม่เคยดึง URL ที่มีการพูดถึง Keyword ใน Pantip

Get list of URL from search page

In [ ]:
#urls = 'https://pantip.com/search?q=นมเมจิ'
#urls = 'https://pantip.com/search?q=นมดัชมิลล์'
#urls = 'https://pantip.com/search?q=นมแดรี่โฮม'
#urls = 'https://pantip.com/search?q=นมโชคชัย'
#urls = 'https://pantip.com/search?q=นมโฟร์โมสต์'

#driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
#driver.get(urls)

Scroll down to get all data (Crawler)

In [ ]:
#last_height = driver.execute_script("return document.body.scrollHeight")
#while True:
#    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#    time.sleep(2)
#    new_height = driver.execute_script("return document.body.scrollHeight")
#    if new_height == last_height:
#        break
#    last_height = new_height

Extract all URLs (web scraping)

In [ ]:
#content=driver.page_source
#soup=BeautifulSoup(content,'lxml')
#urlList = []
# ใช้ class_ เพื่อดึง div class จำเพาะ เพื่อไม่ให้ข้อความอื่นที่ไม่ต้องการปนมา
#for div in soup.find_all("div", class_=lambda value: value and value=="title col-md-12"):
#    for a in div.find_all("a", class_=lambda value: value and value=="datasearch-in", href=True):
#        urlList.append(str(a['href']).strip())

Save URLs into csv file

In [ ]:
#df_urls = pd.DataFrame(urlList)
#df_urls.insert(1,'Retrived date',today)
#df_urls.columns = ['URLs','Retrived date']
#df_urls.to_csv(urls_past_file, index=False)

In [ ]:
#driver.quit()

### กรณีเคยเก็บ URL แล้ว ให้เริ่มตรงนี้ได้เลย เพื่อไม่ให้ต้องทำ Request บ่อยๆ

Loop for scaping by URLs from file

In [112]:
# เรียก URL Meiji ไม่ซ้ำกัน 1000 URLs
df_URLRead = pd.read_csv(urls_past_file[0])
df_URLRead = pd.DataFrame(df_URLRead['URLs'].unique())
df_URLRead.columns = ['URLs']

In [113]:
# เรียก URL จากแบรนด์รองทั้งหมด ดึงพร้อมกัน ไม่ซ้ำกัน 601 URLs
df_URLRead1 = pd.read_csv(urls_past_file[1])
df_URLRead2 = pd.read_csv(urls_past_file[2])
df_URLRead3 = pd.read_csv(urls_past_file[3])
df_URLRead4 = pd.read_csv(urls_past_file[4])
dfURL = pd.concat([df_URLRead1,df_URLRead2,df_URLRead3,df_URLRead4])
dfURL = pd.DataFrame(dfURL['URLs'].unique())
dfURL.columns = ['URLs']

In [151]:
dfURL.head()

,URLs
0,https://pantip.com/topic/39797060
1,https://pantip.com/topic/39337438
2,https://pantip.com/topic/39990528
3,https://pantip.com/topic/39455240
4,https://pantip.com/topic/38874550


In [124]:
# Find Dup URL from meiji
df_dupURL = pd.merge(dfURL,df_URLRead,how='inner')
df_dupURL.tail()

,URLs
74,https://pantip.com/topic/31477185
75,https://pantip.com/topic/31346514
76,https://pantip.com/topic/31198026
77,https://pantip.com/topic/31124656
78,https://pantip.com/topic/30910132


In [128]:
df_URLs = dfURL[~dfURL.URLs.isin(set(df_dupURL.URLs))]

In [129]:
df_URLs.shape

(522, 1)

In [130]:
all_urls = list(df_URLs['URLs'].unique())

In [131]:
len(all_urls)

522

In [149]:
startResult = 0
endResult = len(all_urls)

In [150]:
comments_list = []
for cnt,au in enumerate(all_urls[startResult:endResult],0):
    today = datetime.now()
    print('Processing URL no. ',cnt+1,' on ',today)
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(au)
    content=driver.page_source
    soup=BeautifulSoup(content,'lxml')
    df_comments = pd.DataFrame()
    try:
        for div in soup.find_all("div", class_=lambda value: value and value=="display-post-story"):
            comments_list.append([au,str(div.text).strip()])
        df_comments = pd.DataFrame(comments_list)
        df_comments.insert(2,'Retrived date',today)
        df_comments.columns = ['URLs', 'text', 'Retrived date']
        df_comments.to_csv(comment_past_file[0], index=False, header=False, mode='a')
    except:
        print('กระทู้ถูกลบ')
    comments_list.clear()
    del df_comments
    time.sleep(postNumSleep)
    driver.quit()
    clear_output(wait=True)
    #จำกัดจำนวนกระทู้ สำหรับ Debug
    #if cnt == 3:
    #    break

Processing URL no.  1  on  2020-10-27 13:54:33.920930
